In [13]:
# Import all necessary packages
import pandas as pd
import numpy as np
import gzip

In [14]:
# Get the current working directory
import os
path = os.getcwd()
print(path)

C:\Users\vmoha\documents\Python Data\CSV


In [15]:
os.chdir("C:\\Users\\vmoha\\documents\\Python Data\\CSV")

In [16]:
#Read the csv files to dataframes.
#dfm - metadata for Clothing, Shoes & Jewellery
#dfc - reviews for Clothing, Shoes & Jewellery
#dfe - reviews for Electronics

dfm=pd.read_csv("metadata_category_clothing_shoes_and_jewelry_only.csv",index_col=0)
dfc= pd.read_csv("reviews_Clothing_Shoes_and_Jewelry_5.csv",index_col=0)
dfe=pd.read_csv("reviews_Electronics_5.csv",index_col=0)


In [17]:
#The metadata for "Electronics" is available in the json format.Code provided via http://jmcauley.ucsd.edu/data/amazon/ defines a function to parse the json file to DataFrame.
#dfm2 - Metadata for Electronics

def parse_gz(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)


def convert_to_DF(path):
    i = 0
    df = {}
    for d in parse_gz(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [18]:
dfm2 = convert_to_DF('meta_Electronics.json.gz')
dfm2.rename(columns={'salesRank': 'salesrank'},inplace=True) 

In [19]:
# Convert all the columns of the metadata to string type
dfm= dfm.astype(str)
dfm2= dfm2.astype(str)

In [20]:
#Concatenate the metadata for clothing and electronics create one dataframe
meta=pd.concat([dfm,dfm2])

In [21]:
# Create a field for Category and Sales Rank by splitting the "Salesrank" column
meta[["Category","Sales_Rank"]] = pd.DataFrame(meta.salesrank.str.split(':',1,expand=True))
meta['Category']= meta['Category'].str.replace('{','',regex=True)
meta['Category']= meta['Category'].str.replace('}','',regex=True)
meta['Category']= meta['Category'].str.replace('\'','',regex=True)
meta['Sales_Rank']= meta['Sales_Rank'].str.replace('}','',regex=True)

In [22]:
# Add a column for Reporting Category for dfc and dfe before concatinating to idetify the product category
dfc['Reporting_category'] = pd.Series(["Clothing, Shoes & Jewellery" for x in range(len(dfc.index))])
dfe['Reporting_category'] = pd.Series(["Electronics" for x in range(len(dfe.index))])

In [23]:
#Concatenate dfc and dfe to create one dataframe
df=pd.concat([dfc,dfe])

In [29]:
# Split the 'helful' column to extract "helpful votes" & "Total Votes"
df['helpful']= df['helpful'].str.replace('[','',regex=True)
df['helpful']= df['helpful'].str.replace(']','',regex=True)
df[["helpful_votes","total_votes"]] = pd.DataFrame(df.helpful.str.split(',',1,expand=True))
df["reviewTime"] = pd.to_datetime(df["reviewTime"])

In [32]:
df.total_votes=df.total_votes.astype(int)

In [33]:
df_mean=df.groupby(['asin','reviewTime'])['overall'].mean()

In [34]:
# Create a new column "quality" to categorize ratings. 
# "Positive" for  4-5 star ratings,"Neutral" for 3 star ratings & "bad" for < 3 ratings 
df["quality"]=df.loc[:,"overall"].apply(lambda x:"positive" if x >= 4 else ("neutral" if x==3 else "bad"))

In [35]:
# Create a new column "vote range" to categorize number of votes for the reviews 
df["vote_range"]=df.loc[:,"total_votes"].apply(lambda x:"0 Votes" if x == 0 else ("1-5 Votes" if x<=5 else( "5-10 votes" if  x<=10 else ( "10-50 Votes" if x<=50 else( "50-100 Votes" if x<=100 else "Over 100 Votes")) ) ))

In [36]:
for i, each in enumerate(dfc.overall.value_counts()):
    print(f"Percentgae of {dfc.overall.value_counts().index[i]} starts : {(each*100/len(dfc.overall)):.2f}%")

Percentgae of 5.0 starts : 58.58%
Percentgae of 4.0 starts : 20.94%
Percentgae of 3.0 starts : 10.92%
Percentgae of 2.0 starts : 5.55%
Percentgae of 1.0 starts : 4.02%


In [37]:
for i, each in enumerate(dfe.overall.value_counts()):
    print(f"Percentgae of {dfe.overall.value_counts().index[i]} starts : {(each*100/len(dfe.overall)):.2f}%")

Percentgae of 5.0 starts : 59.73%
Percentgae of 4.0 starts : 20.54%
Percentgae of 3.0 starts : 8.42%
Percentgae of 1.0 starts : 6.44%
Percentgae of 2.0 starts : 4.86%


In [38]:
meta.head()

,asin,salesrank,imurl,categories,title,description,price,related,brand,imUrl,Category,Sales_Rank
2005401,B00004SR8Z,{'Clothing': 1631},http://ecx.images-amazon.com/images/I/41RfWLMD...,"[['Clothing, Shoes & Jewelry', 'Luggage & Trav...",Lewis N. Clark Deluxe Neck Stash,nan,12.8,"{'also_bought': ['B004RJWFDU', 'B00920ZC3O', '...",Lewis N. Clark,NaN,Clothing,1631
2217897,B0000ZE74A,{'Clothing': 4742},http://ecx.images-amazon.com/images/I/41ryA-RO...,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",Vanity Fair Women's Lollipop Plus Size Cuff Le...,nan,nan,"{'also_bought': ['B004PEHJ6U', 'B0000TF5VK', '...",nan,NaN,Clothing,4742
2220611,B00012O12A,{'Clothing': 4},http://ecx.images-amazon.com/images/I/41Dd7rCH...,"[['Clothing, Shoes & Jewelry', 'Men', 'Clothin...",Levi's Men's 505 Regular-Fit Jean,nan,nan,"{'also_bought': ['B0018OR118', 'B003M2X1VI', '...",nan,NaN,Clothing,4
2324985,B0002XSXWC,{'Clothing': 300},http://ecx.images-amazon.com/images/I/41-HS7D7...,"[['Sports & Outdoors', 'Clothing', 'Men', 'Shi...",Columbia Men's Bonehead Short Sleeve Fishing S...,nan,nan,"{'also_bought': ['B0031RFWMU', 'B0050VLSLO', '...",nan,NaN,Clothing,300
2348742,B00066TWMU,{'Clothing': 1199},http://ecx.images-amazon.com/images/I/51UNLUB4...,"[['Clothing, Shoes & Jewelry', 'ASICS'], ['Clo...",ASICS Mesh Backpack,nan,12.41,"{'also_bought': ['B009DB3RS4', 'B001PFA35G', '...",ASICS,NaN,Clothing,1199


In [39]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,Reporting_category,helpful_votes,total_votes,quality,vote_range
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","0, 0",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,2011-02-12,"Clothing, Shoes & Jewellery",0,0,positive,0 Votes
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"0, 0",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,2013-01-19,"Clothing, Shoes & Jewellery",0,0,positive,0 Votes
2,A1RLQXYNCMWRWN,0000031887,Carola,"0, 0",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,2013-01-04,"Clothing, Shoes & Jewellery",0,0,positive,0 Votes
3,A8U3FAMSJVHS5,0000031887,Caromcg,"0, 0","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,2014-04-27,"Clothing, Shoes & Jewellery",0,0,positive,0 Votes
4,A3GEOILWLK86XM,0000031887,CJ,"0, 0",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,2014-03-15,"Clothing, Shoes & Jewellery",0,0,positive,0 Votes


In [143]:
# Write metadata and reviews into a csv for subsequent analysis in Tableau
meta.to_csv(r'C:\\Users\\vmoha\\documents\\Python Data\\CSV\\metadata.csv', index = False)
df.to_csv(r'C:\\Users\\vmoha\\documents\\Python Data\\CSV\\total_reviews.csv', index = False)